In [ ]:
# Librerías
from imblearn.over_sampling import SMOTE  
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cambiar directorio a 'outputs' para leer los CSV
os.chdir("C:/repo personal/PYTHON/deteccion de fraudes/outputs")

# Subir un nivel desde 'modelos/' a la raíz 'deteccion de fraudes/'
sys.path.append(r"C:/repo personal/PYTHON/deteccion de fraudes/scripts")
# Importar la función ajustar_umbral desde utils.py
from utils import ajustar_umbral



In [ ]:

# Cargar datos
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").squeeze()
y_test = pd.read_csv("y_test.csv").squeeze()


In [ ]:
# Aplicar SMOTE para balancear los datos de entrenamiento

smote = SMOTE(random_state=42)  # Crear una instancia de SMOTE
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Datos balanceados con SMOTE:")
print(f"Tamaño original: {X_train.shape}, {y_train.shape}")
print(f"Tamaño balanceado: {X_train_balanced.shape}, {y_train_balanced.shape}")

In [ ]:

# Escalar los datos
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

In [ ]:
# Crear el modelo base
model = LogisticRegression(max_iter=5000, random_state=42)
# Entrenar el modelo
model.fit(X_train_balanced, y_train_balanced)

In [ ]:
# Predicciones en el set de prueba
y_pred = model.predict(X_test)

# También obtenemos probabilidades
y_prob = model.predict_proba(X_test)[:, 1]


In [ ]:
# Definir umbrales
thresholds = np.arange(0.1, 1.0, 0.1)

# Calcular métricas para cada umbral
resultados = ajustar_umbral(y_test, y_prob, thresholds)

# Convertir resultados a un DataFrame para visualización
resultados_df = pd.DataFrame(resultados, columns=["Threshold", "Precision", "Recall", "F1-Score"])

# Mostrar resultados
print(resultados_df)

In [ ]:
# Visualizar métricas en función del umbral
plt.figure(figsize=(10, 6))
plt.plot(resultados_df["Threshold"], resultados_df["Precision"], label="Precision", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["Recall"], label="Recall", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["F1-Score"], label="F1-Score", marker="o")
plt.title("Métricas en función del umbral")
plt.xlabel("Umbral")
plt.ylabel("Valor de la métrica")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Seleccionar el mejor umbral basado en F1-Score
mejor_umbral = resultados_df.loc[resultados_df["F1-Score"].idxmax(), "Threshold"]
print(f"Mejor umbral basado en F1-Score: {mejor_umbral:.2f}")

# Generar predicciones finales con el mejor umbral
y_pred_final = (y_prob >= mejor_umbral).astype(int)


In [ ]:
# Matriz de confusión con el mejor umbral
conf_matrix = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Clase 0", "Clase 1"], yticklabels=["Clase 0", "Clase 1"])
plt.title(f"Matriz de Confusión (Umbral = {mejor_umbral:.2f})")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

In [ ]:
# Reporte de clasificación con el mejor umbral
print("Reporte de Clasificación con el mejor umbral:")
print(classification_report(y_test, y_pred_final, digits=4))

In [ ]:
# Calcular Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
# Imprimir métricas clave
print(f"PR-AUC: {pr_auc:.4f}")

In [ ]:
# Visualizar curva Precision-Recall
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}")
plt.title("Curva Precision-Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.grid()
plt.show()